# Trend Analysis

## Load Workspace

In [ ]:
from matplotlib import pyplot as plt

from scipy import stats as sts

import numpy as np

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format

import os
os.makedirs('csv-output', exist_ok = True)
os.makedirs('png-output', exist_ok = True)

## Input Filename

In [ ]:
filename = 'input'
filename += '.csv'

## Input Dataframe

In [ ]:
input_data = pd.read_csv(filename, header = None)
input_data.head()

## Data Linear Regression

In [ ]:
funxy = pd.DataFrame([np.arange(1, len(input_data) + 1), 
                      input_data.iloc[:, 0]]).transpose()

original_reg = sts.linregress(funxy.iloc[:, 0], funxy.iloc[:, 1])

reg_line = pd.DataFrame(original_reg.intercept + original_reg.slope * funxy.iloc[:, 0])
reg_line.to_csv('csv-output/01_data_linear_reg.csv', header = False, index = False)
reg_line.head()

## Preparing Canvas

In [ ]:
plt.rcParams['figure.figsize'] = 13, 8
plt.rcParams['font.family'] = 'Georgia'
plt.rcParams['font.style'] = 'italic'
plt.style.use('ggplot')

## Original Data Trend

In [ ]:
ax = plt.axes()
ax.minorticks_on()
ax.set_title('Original Data Trend', fontsize = '42', pad = 20)

plt.xlabel('Month', fontsize = '28', labelpad = 18)
plt.ylabel('Station Measurement', fontsize = '28', labelpad = 18)

for item in (ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

plt.plot(funxy.iloc[:, 0], funxy.iloc[:, 1], marker = 'o', markersize = 4.5,
         linestyle = ':', color = '#28BBA5',label = 'Data')

plt.plot(funxy.iloc[:, 0], reg_line, linestyle = '--', color = '#D4AA2A',
         label = 'Trendline')

leg = plt.legend(prop = {'size':16}, loc = 'center left', 
                 bbox_to_anchor = (1, 0.5))

for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

plt.tight_layout()
plt.savefig('png-output/01_original_data_trend.png', bbox_inches = 'tight')
plt.show()

## Trend in Mean

In [ ]:
Am = original_reg.intercept
Bm = original_reg.slope
r = original_reg.rvalue

tc = r * np.sqrt(len(input_data) - 2)
tc /= np.sqrt(1 - r ** 2)

tt = sts.t.ppf(.975, len(input_data) - 2)

tm_results = pd.DataFrame([[Am, Bm, r], [tc, tt]]).transpose()
tm_results.to_csv('csv-output/02_trend_mean_results.csv', header = False, index = False)
tm_results.head()


## Removal of Trend in Mean

In [ ]:
Yt = input_data.sub(reg_line)
Yt.to_csv('csv-output/03_Yt.csv', header = False, index = False)
Yt.head()

## Standard Deviation Regression

In [ ]:
Yt_std_N = len(input_data) // 12
Yt_std = pd.DataFrame(range(Yt_std_N))

for i in range(Yt_std_N):
    Yt_std.iloc[i] = float(Yt.loc[12 * i:11 + 12 * i].std())

Yt_std_reg = sts.linregress(np.arange(1, Yt_std_N + 1), Yt_std.transpose())

Yt_std.to_csv('csv-output/04_Yt_std_dev.csv', header = False, index = False)
Yt_std.head()

## Trend in Standard Deviation

In [ ]:
As = Yt_std_reg.intercept
Bs = Yt_std_reg.slope
r = Yt_std_reg.rvalue

tc = r * np.sqrt(len(Yt_std) - 2)
tc /= np.sqrt(1 - r ** 2)

tt = sts.t.ppf(.975, len(Yt_std) - 2)

std_results = pd.DataFrame([[As, Bs, r], [tc, tt]]).transpose()
std_results.to_csv('csv-output/05_trend_std_results.csv', header = False, index = False)
std_results.head()

## Standard Deviation Trend

In [ ]:
stdreg_line = pd.DataFrame(Yt_std_reg.intercept + Yt_std_reg.slope * np.arange(1, Yt_std_N + 1))
stdreg_line.to_csv('csv-output/06_std_linear_reg.csv', header = False, index = False)

ax = plt.axes()
ax.set_title('Standard Deviation Trend', fontsize = '42', pad = 20)

plt.xlabel('Year', fontsize = '28', labelpad = 18)
plt.ylabel('Standard Deviation', fontsize = '28', labelpad = 18)

for item in (ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

plt.plot(np.arange(1, Yt_std_N + 1), Yt_std.iloc[:,0], marker = 'o', markersize = 4.5,
         linestyle = '', color = '#712EAD',label = 'Original Data')

plt.plot(range(1, Yt_std_N + 1), stdreg_line.iloc[:, 0], linestyle = '--', color = '#71A5CB',
         label = 'Trendline')

leg = plt.legend(prop = {'size':16}, loc = 'center left', 
                 bbox_to_anchor = (1, 0.5))

for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

plt.tight_layout()
plt.savefig('png-output/02_std_dev_trend.png', bbox_inches = 'tight')
plt.show()

## Removal of Trend in Standard Deviation

In [ ]:
Ts = pd.DataFrame(np.zeros(len(Yt)))

for i in range(Yt_std_N):
    Ts.iloc[12 * i: 12 + 12 * i] += As + Bs * (i + 1)

Zt = Yt.div(Ts)
Zt.to_csv('csv-output/07_Zt.csv', header = False, index = False)
Zt.head()

## Data Correction

In [ ]:
output_data = input_data.mean() + input_data.std() * Zt
output_data.to_csv('csv-output/08_data_correction.csv', header = False, index = False)
output_data.head()

## Final Data Trend

In [ ]:
final_reg = sts.linregress(funxy.iloc[:, 0], output_data.iloc[:,0])
final_reg_line = pd.DataFrame(final_reg.intercept + final_reg.slope * funxy.iloc[:, 0])
final_reg_line.to_csv('csv-output/09_final_linear_reg.csv', header = False, index = False)

ax = plt.axes()
ax.minorticks_on()
ax.set_title('Data Trend', fontsize = '42', pad = 20)

plt.xlabel('Month', fontsize = '28', labelpad = 18)
plt.ylabel('Station Measurement', fontsize = '28', labelpad = 18)

ax = plt.axes()
ax.minorticks_on()
ax.set_title('Final Data Trend', fontsize = '42', pad = 20)

plt.ylabel('Corrected Measurement', fontsize = '28', labelpad = 18)
plt.xlabel('Month', fontsize = '28', labelpad = 18)

for item in (ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

plt.plot(funxy.iloc[:, 0], output_data, marker = 'o', markersize = 4.5,
         linestyle = ':', color = '#28BBA5', label = 'Final Data')

plt.plot(funxy.iloc[:, 0], final_reg_line, linestyle = '--', color = '#D4AA2A',
         label = 'Trendline')

leg = plt.legend(prop = {'size':16}, loc = 'center left', 
                 bbox_to_anchor = (1, 0.5))

for line, text in zip(leg.get_lines(), leg.get_texts()):
    text.set_color(line.get_color())

plt.tight_layout()
plt.savefig('png-output/03_final_data_trend.png', bbox_inches = 'tight')
plt.show()